<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA/blob/main/lesson_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие №2
# Python и библиотеки для анализа данных и математического моделирования

## Установка Python

![anaconda image](https://www.anaconda.com/wp-content/uploads/2022/12/anaconda_secondary_logo.svg)

**Anaconda** — дистрибутив языков программирования Python и R, включающий набор популярных свободных библиотек, объединённых проблематиками науки о данных и машинного обучения. Основная цель — поставка единым согласованным комплектом наиболее востребованных соответствующим кругом пользователей тематических модулей (таких как NumPy, SciPy, Astropy и других) с разрешением возникающих зависимостей и конфликтов, которые неизбежны при одиночной установке.

Программа Anaсonda представляет собой полноценную систему работы с библиотеками Python, включающую программу-установщик и набор следующих файлов.

* **Библиотеки.** Каждая из них включает определенный набор функций и объектов, предназначенный для решения конкретных задач — например, работы с массивами данных, астрономических вычислений, обработки изображений, построения и тренировки нейронных сетей, различных инженерных расчетов, статистического анализа и т.д. Всего в пакет Anaconda входит около 1500 библиотек, объединенных научной и инженерной тематикой.
* **Менеджер пакетов Conda.** Это кроссплатформенное программное обеспечение, предназначенное для установки пакетов ПО и библиотек, быстрого переключения между их версиями, загрузки и инсталляции обновлений. Conda написана на Python, но является языково-независимой, то есть может работать с проектами, написанными на любом языке программирования. Кроме того, с ее помощью можно установить и сам Python — возможность, предусмотренная далеко не во всех менеджерах пакетов, написанных на этом ЯП. Из-за того, что менеджер пакетов оказался очень удобен для решения задач, не связанных с Python, он был выделен в отдельный дистрибутив с открытым исходным кодом.
* **Менеджер виртуальной среды.** Это программа, позволяющая создавать, редактировать, удалять и отслеживать состояние виртуальных машин. Управление изолированными виртуальными средами осуществляется из единой контрольной консоли. Виртуализация используется при работе над несколькими проектами одновременно, позволяя исключить их влияние друг на друга, тем самым обеспечив высокую точность расчетов.
* **GIU Anaconda Navigator.** GUI (graphic user interface) — графический интерфейс пользователя, то есть программная оболочка, в которой можно работать с библиотеками не с помощью текстовых команд, а используя визуальные элементы, такие как блоки, модули, графики и т.д. «Анаконда Навигатор» существенно упрощает взаимодействие с библиотеками, делает работу менее однообразной и утомительной, повышает наглядность рабочего процесса.

https://www.anaconda.com/

## Установка виртуального окружения
С помощью виртуального окружения можно создать изолированный контейнер с нужными версиями под каждую задачу.  
```conda env create -f environment.yml```

## Инструмент разработки

![jupyter image](https://jupyter.org/assets/logos/rectanglelogo-greytext-orangebody-greymoons.svg)

**Jupyter Notebok** — браузерный инструмент для оформления красивой и информативной аналитики, совместного хранения кода, картинок, примечаний, диаграмм, формул и т.д.

В мире Data Science Jupyter-ноутбук уже несколько лет считается одним из популярных инструментов для анализа данных и быстрого прототипирования.

Один из плюсов этого инструмента в том, что код можно разделить на кусочки и работать над ними в любом порядке. Например, написать скрипт и сразу посмотреть, как он работает. Остальные фрагменты кода при этом запускать не нужно, результат появляется тут же, под частью кода.

```$ jupyter notebook```


<!-- ![colab image](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Google_Colaboratory_SVG_Logo.svg/2560px-Google_Colaboratory_SVG_Logo.svg.png width="100") -->

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Google_Colaboratory_SVG_Logo.svg/2560px-Google_Colaboratory_SVG_Logo.svg.png" alt="numpy image" width="200"/>

**Google colab** - это облачный сервис от Google, который позволяет писать и выполнять Python-код в браузере без установки дополнительных программ.

Главные преимущества:

https://colab.research.google.com/rch.google.com/

## Библиотеки

<!-- ![numpy image](https://numpy.org/images/logo.svg) -->
<img src="https://numpy.org/images/logo.svg" alt="numpy image" width="100"/>

**NumPy** это open-source модуль для python, который предоставляет общие математические и числовые операции в виде пре-скомпилированных, быстрых функций. Они объединяются в высокоуровневые пакеты. Они обеспечивают функционал, который можно сравнить с функционалом MatLab. NumPy (Numeric Python) предоставляет базовые методы для манипуляции с большими массивами и матрицами.

*Входит в дистрибутив Anaconda*

<!-- ![pandas image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKPePf0eI9lrP20Ym-P0v-_B2yB6IHRoQiWw&s) -->
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKPePf0eI9lrP20Ym-P0v-_B2yB6IHRoQiWw&s" alt="pandas image" width="200"/>

**Pandas** — программная библиотека на языке Python для обработки и анализа данных. Работа pandas с данными строится поверх библиотеки NumPy, являющейся инструментом более низкого уровня. Предоставляет специальные структуры данных и операции для манипулирования числовыми таблицами и временны́ми рядами. Название библиотеки происходит от эконометрического термина «панельные данные», используемого для описания многомерных структурированных наборов информации.

*Входит в дистрибутив Anaconda*

![sklearn image](https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png)

**Scikit-learn** — библиотека, предназначенная для машинного обучения, написанная на языке программирования Python и распространяемая в виде свободного программного обеспечения.

В её состав входят различные алгоритмы, в том числе предназначенные для задач классификации, регрессионного и кластерного анализа данных, включая метод опорных векторов, метод случайного леса, алгоритм усиления градиента, метод k-средних и DBSCAN. Библиотека была разработана для взаимодействия с численными и научными библиотеками языка программирования Python NumPy и SciPy.

*Входит в дистрибутив Anaconda*

<!-- ![pytorch image](https://upload.wikimedia.org/wikipedia/commons/9/96/Pytorch_logo.png) -->
<img src="https://upload.wikimedia.org/wikipedia/commons/9/96/Pytorch_logo.png" alt="pytorch image" width="200"/>

**PyTorch** — фреймворк машинного обучения для языка Python с открытым исходным кодом, созданный на базе Torch. Используется для решения различных задач: компьютерное зрение, обработка естественного языка и т.д. Разрабатывается преимущественно группой искусственного интеллекта Facebook.

```conda install pytorch::pytorch```

<!-- ![matplotlib image](https://matplotlib.org/_static/logo_light.svg) -->
<img src="https://matplotlib.org/_static/logo_light.svg" alt="matplotlib image" width="200"/>

**Matplotlib** — библиотека на языке программирования Python для визуализации данных двумерной и трёхмерной графикой. Получаемые изображения могут быть использованы в качестве иллюстраций в публикациях.

*Входит в дистрибутив Anaconda*

<img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Plotly-logo.png" alt="plotly image" width="200"/>

**Plotly** — библиотека для визуализации данных. Plotly позволяет строить интерактивные визуализации. С помощью Plotly можно как изучать какие-то данные «на лету» (не перестраивая график в matplotlib, изменяя масштаб, включая/выключая какие-то данные), так и построить полноценный интерактивный отчёт (дашборд).

*Входит в дистрибутив Anaconda*